# Datenaustausch WIAG/FactGrid, Teil 1
## Lies FactGrid-IDs in WIAG ein.

Wenn IDs im FactGrid manuell recherchiert worden sind und in WIAG eingetragen wurden, dann ist zu prüfen, ob zunächst die entsprechenden WIAG-IDs auch manuell in FactGrid eingetragen werden sollen.

- Lies die aktuellen FQ-IDs aus WIAG ein. 
- Lies die aktuellen WIAG-IDs aus FactGrid ein.
- Prüfe in Bezug auf die WIAG-ID auf doppelte Einträge in WIAG, in FactGrid und auf unterschiedliche Zuordnungen in WIAG und FactGrid
- Entferne Einträge, die im FactGrid schon eine ID haben.

In [ ]:
using DataFrames, CSV

In [ ]:
input_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

In [ ]:
output_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data_sql"

## Lies und prüfe die IDs aus FactGrid

Frage IDs im FactGrid ab (SPARQL) und speichere das Ergebnis in einer `*.tsv`-Datei.  
https://database.factgrid.de/query/


``` sparql
SELECT DISTINCT ?item ?itemLabel ?wiag
	WHERE
	{
	  ?item wdt:P601 ?wiag.
	  ?item wdt:P2 wd:Q7
	    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
	}    

```

Lies die Daten ein. Gib die Zahl der Einträge aus.

In [ ]:
input_file = "FactGrid-IDs_2023-12-18.tsv"
input_path_file = joinpath(input_path, input_file)

In [ ]:
df_fq_ids = CSV.read(input_path_file, DataFrame, delim = "\t");
nrow(df_fq_ids)

Kontroll-Ausgabe

In [ ]:
df_fq_ids[17:22, :]

Extrahiere die Q-Nummer aus der Spalte `item`.

In [ ]:
find_fq_id(s) = last(split(s, "/"))

In [ ]:
transform!(df_fq_ids, :item => ByRow(find_fq_id) => :q_id);

Finde doppelte Einträge

In [ ]:
df_fq_m = groupby(df_fq_ids, :wiag);
size(df_fq_m)

In [ ]:
df_fq_m_count = combine(df_fq_m, nrow => :n);

Zeige, wo die WIAG-Nummer mehrfach erscheint.

In [ ]:
df_fq_m_2 = subset(df_fq_m_count, :n => ByRow(n -> n > 1))

Falls es Mehrfacheinträge gibt: Erhalte mehr Informationen durch einen Filter für diese Einträge. Die angegebenen WIAG-Nummern sind Beispiele, welche durch die WIAG-Nummern der vorangegangenen Ausgabe ersetzt werden müssen.

In [ ]:
wiag_mult = [
    "WIAG-Pers-CANON-11582-001", 
    "WIAG-Pers-CANON-15754-001"
]

In [ ]:
crit_wiag(s) = s in wiag_mult
subset(df_fq_ids, :wiag => ByRow(crit_wiag))

Falls es Mehrfacheinträge gibt: Übernimm nur Zeilen ohne die jeweils überzähligen FactGrid-IDs. Die Filterliste enthält IDs, die **nicht** in WIAG übernommen werden sollen.

In [ ]:
crit_fq(s) = s in ["Q646386", "Q646394"]

In [ ]:
df_fq_ids = subset(df_fq_ids, :q_id => ByRow(!crit_fq));
nrow(df_fq_ids)

Die Ausgabe der folgenden Zelle sollte keine Mehrfacheinträge mehr enthalten.

In [ ]:
subset(df_fq_ids, :wiag => ByRow(crit_wiag))

## Lies und prüfe die FactGrid-IDs aus WIAG

Exportiere Personendaten aus WIAG: "Export"/"CSV Personendaten"  
Letzer Export: Server LIVE, Datum 2023-12-18 12:20

Lies die Personendaten aus WIAG ein.

In [ ]:
input_file = "WIAG-can_2023-12-18.csv"
input_path_file = joinpath(input_path, input_file)

In [ ]:
df_p = CSV.read(input_path_file, DataFrame);
nrow(df_p)

Wähle relevante Spalten aus.

In [ ]:
df_p_fq = select(df_p, [:id, :FactGrid_ID, :corpus, :displayname]);

## Vergleiche die beiden Quellen

In [ ]:
df_fq = outerjoin(df_p_fq, df_fq_ids, on = :id => :wiag);
nrow(df_fq)

Die Zahl ist größer als die Zahl der Einträge in `df_p_fq` aus WIAG, weil in der Liste aus FactGrid die Bischöfe auch enthalten sind.

In [ ]:
names(df_fq)

Ermittle die Einträge mit FactGrid-ID in WIAG aber ohne Eintrag in FactGrid.

In [ ]:
a_mg_b(a, b) = !ismissing(a) && ismissing(b)

In [ ]:
df_p_only = subset(df_fq, [:FactGrid_ID, :q_id] => ByRow(a_mg_b))

Ermittle die Einträge mit unterschiedlichen Werten für die FactGrid-ID

In [ ]:
missing_or_equal(a, b) = ismissing(a) || ismissing(b) || a == b

In [ ]:
df_diff = subset(df_fq, [:FactGrid_ID, :q_id] => ByRow(!missing_or_equal))

Sortiere die Bischöfe wieder aus. (Nach der Prüfung auf ungleiche IDs!) 

In [ ]:
is_gatz_id(s) = occursin("EPISCGatz", s)

In [ ]:
subset!(df_fq, :id => ByRow(!is_gatz_id))
nrow(df_fq)

Zahl der neu einzutragenden FactGrid-IDs: Einträge mit einer ID im FactGrid, aber nicht in WIAG

In [ ]:
df_insert = subset(df_fq, [:q_id, :FactGrid_ID] => ByRow(a_mg_b));
nrow(df_insert)

Beispiel für das SQL-insert Statement, das in der Folge automatisch für jede Zeile in `df_insert` erzeugt wird.
``` sql
INSERT INTO url_external (item_id, value, authority_id)
	    SELECT item_id, 'Q648993', 42 FROM item_corpus
	    WHERE id_public = "WIAG-Pers-EPISCGatz-02525-001";
```

In [ ]:
function make_insert_stmt(fq_id, wiag_id)
    return """
    INSERT INTO url_external (item_id, value, authority_id)
    SELECT item_id, '$(fq_id)', 42 FROM item_corpus
    WHERE id_public = "$(wiag_id)";
    """
end

Schreibe eine Datei mit SQL-Statements, mit der die FactGrid IDs in die Tabelle `url_external` eingelesen werden können.

In [ ]:
output_file = "insert-uext-can_2023-12-18.sql"
output_file_path = joinpath(output_path, output_file)

In [ ]:
open(output_file_path, "w") do out
    println(out, "LOCK TABLES url_external WRITE, item_corpus WRITE;")
    for row in eachrow(df_insert)
        println(out, make_insert_stmt(row[:q_id], row[:id]))
    end
    println(out, "UNLOCK TABLES;")
end


Lies die entstandene Datei in die WIAG-Datenbank ein. Die Datei kann z.B. über phpMyAdmin in eine Datenbank eingelesen werden. Nachdem die Datenbank dort ausgewählt ist, wählt man im Menu den Punkt "Import", setzt die Datei ein über "Durchsuchen" und startet den Import mit "Ok".